# Finder Chart for each star

In [126]:
%pylab
%matplotlib inline

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [127]:
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
# import webbrowser

In [128]:
def finder_chart_url(ra=62.165336, dec=47.712509, 
                     v='poss2ukstu_red',
                     epoch='J2000',
                     h=5.,
                     w=5.,
                     f='gif'):
    """ return the url of stsci DSS
    
    Parameters
    ----------
    
    ra: right ascension
        number/array. range: [0.0 360.0]
    
    dec: declination
        number/array. range: [-90.0 +90.0]
    
    v3: string
        {
        'poss2ukstu_red', 'poss2ukstu_blue', 'poss2ukstu_ir'
        'poss1_red', 'poss1_blue',
        'quickv',
        'phase2_gsc2', 'phase2_gsc1'
        }
    
    epoch: string {'J2000' 'B1950'}
        epoch of coordinate system
        
    h,w: number
        height and width of image. maximum is 60. (arcmin)
    
    f: string {'gif' 'fits'}
        file format
    
    Returns
    -------
    
    urlstr: string
        the link of the image
    """
    
    from astropy.coordinates import SkyCoord
    ra, dec = np.array(ra), np.array(dec)
    c = SkyCoord(ra, dec, unit="deg")

    urlform = ['https://archive.stsci.edu/cgi-bin/dss_search?v=%s',
               '&r=%02d+%02d+%f',
               '&d=%s%02d+%02d+%f',
               '&e=%s',
               '&h=%f',
               '&w=%f',
               '&f=%s',
               '&c=none&fov=NONE&v3=']
    urldict = {True:'%2B',False:'-'}
    
    if isscalar(c.ra.hms.h):
        # only 1 pair of [ra, dec]
        urlpiece = [
            urlform[0]%v, 
            urlform[1]%(c.ra.hms.h, c.ra.hms.m, c.ra.hms.s),
            urlform[2]%(urldict[c.dec.dms.d>=0],
                        np.abs(c.dec.dms.d),
                        np.abs(c.dec.dms.m),
                        np.abs(c.dec.dms.s)),
            urlform[3]%epoch,
            urlform[4]%h,
            urlform[5]%w,
            urlform[6]%f,
            urlform[7]
        ]
        urlstr = ''.join(urlpiece)
        print c.dec.dms.m
        return urlstr
    else:# a list of pairs of [ra, dec]
        urlstr = list()
        for c_ in c:
            urlpiece = [
                urlform[0]%v, 
                urlform[1]%(c_.ra.hms.h, c_.ra.hms.m, c_.ra.hms.s),
                urlform[2]%(urldict[c_.dec.dms.d>=0],
                            np.abs(c_.dec.dms.d),
                            np.abs(c_.dec.dms.m),
                            np.abs(c_.dec.dms.s)),
                urlform[3]%epoch,
                urlform[4]%h,
                urlform[5]%w,
                urlform[6]%f,
                urlform[7]
            ]
            urlstr.append(''.join(urlpiece))
        urlstr = np.array(urlstr)
    return urlstr

# urls = finder_chart_url([20.254, 20.255], [-31.354 , 65.15])
# print urls

In [129]:
t = Table.read('../data/w1970.targeted.fits')
c = SkyCoord(t['RAJ2000_2mass'].data, t['DEJ2000_2mass'].data, unit="deg")
urls = finder_chart_url(t['RAJ2000_2mass'].data, t['DEJ2000_2mass'].data,h=15,w=15)

In [130]:
len(t)

80

# Auto-generate finder chart for each star

In [139]:
for i in xrange(len(t)):
    print 'ID: %d \n'%t['id'][i]
    print 'Position: \t[  ra,  dec] = [**%s**, **%s**]\n' % (c.ra[0], c.dec[0])
    print 'Position: \t[  ra,  dec] = [%02d:%02d:%02f, %02d:%02d:%02f]\n' \
        % (c.ra.hms.h[0], c.ra.hms.m[0], c.ra.hms.s[0], c.dec.dms.d[0], c.dec.dms.m[0], c.dec.dms.s[0])
    print 'Position: \t[glon, glat] = [%s, %s]\n' % (c.galactic.l.deg[0], c.galactic.b.deg[0])
    print 'USNO-B1.0 Epoch: %s\n'% t['Epoch'][i]
    print 'Proper motion: (%f, %f) +- (%f, %f)\n'\
        % (t['pmRA_usnob1'][i],t['pmDE_usnob1'][i],t['e_pmRA'][i],t['e_pmDE'][i])
    print '[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=%s&submit=SIMBAD+search): ''%s'' (%s) \n'\
        % (t['Name'][i],t['Name'][i],t['otype'][i])
    print 'USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [%.3f **%.3f** %.3f %.3f %.3f] mag\n' \
        % (t['Bmag_usnob1'][i],t['Rmag_usnob1'][i],t['Jmag'][i],t['Hmag'][i],t['Kmag'][i])
    print 'Exposure Time: **%d min**' % t['tobs'][i]
    print '![](%s)\n'%urls[i]
    print '---\n'


ID: 1 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-4.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  9097&submit=SIMBAD+search):  HR  9097 (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.470 **6.130** 5.410 5.336 5.269] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=00+03+25.717200&d=%2B63+38+25.756800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 2 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad

![](http://nicadd.niu.edu/~donna/PHYS162H/Cluster%20observing/Cluster_chart)

ID: 1 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-4.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  9097&submit=SIMBAD+search):  HR  9097 (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.470 **6.130** 5.410 5.336 5.269] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=00+03+25.717200&d=%2B63+38+25.756800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 2 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 10   CAS&submit=SIMBAD+search):  10   CAS (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.520 **5.570** 5.463 5.470 5.468] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=00+06+26.501520&d=%2B64+11+46.255200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 3 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=KAP   CAS&submit=SIMBAD+search): KAP   CAS (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.240 **4.120** 4.141 4.148 4.013] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=00+33+0.004320&d=%2B62+55+54.210000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 4 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=OMI   CAS&submit=SIMBAD+search): OMI   CAS (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.395 **4.495** 4.653 4.612 4.404] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=00+44+43.512240&d=%2B48+17+3.746400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 5 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, -12.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=PHI   AND&submit=SIMBAD+search): PHI   AND (Star) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.200 **4.320** 4.401 4.451 4.585] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=01+09+30.141840&d=%2B47+14+30.321600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 6 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, -2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=&submit=SIMBAD+search):  (Em*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [7.590 **6.425** 5.381 4.692 4.509] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=01+39+7.145760&d=%2B61+04+44.767200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 7 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (22.000000, -16.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=PHI   PER&submit=SIMBAD+search): PHI   PER (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [3.890 **4.065** 4.049 3.955 3.709] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=01+43+39.646320&d=%2B50+41+19.298400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 8 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-2.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR   618&submit=SIMBAD+search):  HR   618 (RSG*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.135 **5.405** 4.062 4.095 3.646] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=02+08+40.630800&d=%2B58+25+24.736800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 9 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-2.000000, -2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR   654&submit=SIMBAD+search):  HR   654 (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.660 **6.395** 5.776 5.705 5.610] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=02+16+51.728400&d=%2B57+03+18.824400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 10 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-2.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=&submit=SIMBAD+search):  (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.910 **6.395** 5.558 5.426 5.295] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=02+19+4.441440&d=%2B57+08+7.821600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 11 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 10   PER&submit=SIMBAD+search):  10   PER (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.445 **6.150** 5.570 5.489 5.416] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=02+25+16.038240&d=%2B56+36+35.316000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 12 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (18.000000, -28.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR   894&submit=SIMBAD+search):  HR   894 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.035 **6.135** 6.127 6.130 6.104] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=03+00+11.879040&d=%2B38+07+54.307200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 13 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=&submit=SIMBAD+search):  (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.675 **6.380** 5.609 5.489 5.255] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=03+08+54.176400&d=%2B62+23+4.416000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 14 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (12.000000, -16.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR   985&submit=SIMBAD+search):  HR   985 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.580 **4.770** 5.048 5.074 5.048] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=03+19+59.233680&d=%2B65+39+8.604000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 15 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  1040&submit=SIMBAD+search):  HR  1040 (RSG*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.950 **4.325** 3.050 2.876 2.626] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=03+29+54.745920&d=%2B58+52+43.402800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 16 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (18.000000, -30.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=PSI   PER&submit=SIMBAD+search): PSI   PER (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.225 **4.325** 4.316 4.153 4.107] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=03+36+29.367360&d=%2B48+11+33.388800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 17 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (24.000000, -30.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  1113&submit=SIMBAD+search):  HR  1113 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.440 **5.575** 5.617 5.721 5.687] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=03+41+7.843200&d=%2B37+34+48.612000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 18 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (4.000000, -14.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 13   TAU&submit=SIMBAD+search):  13   TAU (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.630 **5.680** 5.635 5.649 5.641] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=03+42+18.952320&d=%2B19+42+1.065600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 19 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (18.000000, -48.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 23   TAU&submit=SIMBAD+search):  23   TAU (PulsV*bCep) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.070 **4.190** 4.233 4.309 4.220] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=03+46+19.584480&d=%2B23+56+54.171600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 20 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-8.000000, -4.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  1160&submit=SIMBAD+search):  HR  1160 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.205 **6.175** 5.949 5.947 5.940] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=03+48+18.058080&d=%2B50+44+12.181200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 21 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (20.000000, -48.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=*BU   TAU&submit=SIMBAD+search): *BU   TAU (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.925 **5.080** 5.087 5.067 4.937] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=03+49+11.213760&d=%2B24+08+12.030000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 22 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  1204&submit=SIMBAD+search):  HR  1204 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.860 **4.980** 5.040 5.128 5.040] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=03+57+25.420320&d=%2B63+04+20.182800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 23 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=*MX   PER&submit=SIMBAD+search): *MX   PER (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [3.930 **4.015** 3.978 3.899 3.796] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=04+08+39.680640&d=%2B47+42+45.032400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 24 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (12.000000, -26.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  1500&submit=SIMBAD+search):  HR  1500 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.115 **6.070** 5.756 5.780 5.716] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=04+44+13.022880&d=%2B40+47+11.191200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 25 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=ALF   CAM&submit=SIMBAD+search): ALF   CAM (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.265 **4.295** 4.243 4.380 4.242] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=04+54+2.979360&d=%2B66+20+33.626400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 26 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 11   CAM&submit=SIMBAD+search):  11   CAM (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.070 **5.260** 5.424 5.021 4.906] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=05+06+8.460960&d=%2B58+58+20.726400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 27 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (2.000000, -8.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=105   TAU&submit=SIMBAD+search): 105   TAU (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.965 **5.755** 5.197 4.961 4.775] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=05+07+55.434720&d=%2B21+42+17.355600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 28 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, -2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=&submit=SIMBAD+search):  (Em*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [7.295 **6.375** 5.153 4.813 4.588] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=05+17+15.465600&d=%2B28+54+18.028800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 29 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 25   ORI&submit=SIMBAD+search):  25   ORI (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.620 **4.950** 5.369 5.417 5.355] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=05+24+44.825760&d=%2B01+50+47.281200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 30 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-6.000000, -12.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  1847&submit=SIMBAD+search):  HR  1847 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.020 **6.070** 5.964 6.022 6.016] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=05+32+14.139120&d=%2B17+03+29.221200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 31 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=120   TAU&submit=SIMBAD+search): 120   TAU (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.620 **5.670** 5.160 5.065 4.777] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=05+33+31.627440&d=%2B18+32+24.860400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 32 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, -2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=OMG   ORI&submit=SIMBAD+search): OMG   ORI (Cl*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.370 **4.560** 5.012 4.924 4.805] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=05+39+11.155200&d=%2B04+07+17.274000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 33 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (6.000000, -16.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  1961&submit=SIMBAD+search):  HR  1961 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.120 **6.255** 5.771 5.803 5.703] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=05+43+19.499760&d=%2B23+12+15.829200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 34 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (2.000000, -6.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=CHI 2 ORI&submit=SIMBAD+search): CHI 2 ORI (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.810 **4.530** 4.186 4.039 3.949] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=06+03+55.201680&d=%2B20+08+18.387600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 35 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, -2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=  3   GEM&submit=SIMBAD+search):   3   GEM (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.905 **5.695** 5.393 5.207 5.120] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=06+09+43.980000&d=%2B23+06+48.452400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 36 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, -18.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  2231&submit=SIMBAD+search):  HR  2231 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.885 **6.095** 6.285 6.336 6.381] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=06+15+40.177200&d=%2B06+03+58.244400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 37 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-6.000000, -14.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= NU   GEM&submit=SIMBAD+search):  NU   GEM (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [3.970 **4.160** 4.585 4.523 4.346] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=06+28+57.787920&d=%2B20+12+43.819200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 38 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (2.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  2370&submit=SIMBAD+search):  HR  2370 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.990 **6.180** 6.042 5.973 5.782] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=06+31+9.557280&d=%2B11+15+5.011200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 39 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-4.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  2422&submit=SIMBAD+search):  HR  2422 (SB) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.055 **6.070** 5.827 5.806 5.714] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=06+37+24.031680&d=%2B06+08+7.425600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 40 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  2467&submit=SIMBAD+search):  HR  2467 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.235 **6.390** 6.444 6.509 6.512] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=06+41+59.232720&d=%2B06+20+43.468800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 41 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-6.000000, 6.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=PSI 9 AUR&submit=SIMBAD+search): PSI 9 AUR (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.730 **5.865** 5.908 5.977 5.986] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=06+56+32.062560&d=%2B46+16+26.407200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 42 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-20.000000, -8.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=THE   CRB&submit=SIMBAD+search): THE   CRB (Star) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.000 **4.220** 4.429 4.472 4.430] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=15+32+55.788000&d=%2B31+21+33.152400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 43 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-4.000000, -12.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=**R   CRB&submit=SIMBAD+search): **R   CRB (Erupt*RCrB) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.355 **5.540** 5.364 5.089 4.564] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=15+48+34.407840&d=%2B28+09+24.264000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 44 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-26.000000, 16.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=  4   HER&submit=SIMBAD+search):   4   HER (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.610 **5.785** 5.821 5.887 5.850] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=15+55+30.591360&d=%2B42+33+58.280400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 45 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (4.000000, 4.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=V441  HER&submit=SIMBAD+search): V441  HER (semi-regV*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.740 **5.265** 4.998 4.239 3.632] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=17+55+25.182000&d=%2B26+02+59.964000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 46 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (2.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  6720&submit=SIMBAD+search):  HR  6720 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.335 **6.435** 6.507 6.566 6.552] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=18+00+27.661680&d=%2B19+30+20.840400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 47 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, -14.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  6873&submit=SIMBAD+search):  HR  6873 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.065 **6.165** 6.128 6.171 6.156] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=18+21+28.405680&d=%2B05+26+8.638800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 48 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (2.000000, 8.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  6984&submit=SIMBAD+search):  HR  6984 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.940 **6.130** 6.063 6.092 6.025] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=18+35+13.491840&d=%2B34+27+28.951200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 49 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (8.000000, -6.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  7084&submit=SIMBAD+search):  HR  7084 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.750 **5.925** 5.870 5.805 5.786] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=18+46+43.092960&d=%2B52+59+16.684800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 50 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-6.000000, -14.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 64   SER&submit=SIMBAD+search):  64   SER (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.535 **5.550** 5.425 5.455 5.390] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=18+57+16.586640&d=%2B02+32+7.299600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 51 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, -8.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=&submit=SIMBAD+search):  (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [7.770 **6.310** 4.179 nan 3.517] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=19+27+26.585760&d=%2B18+17+45.304800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 52 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-2.000000, -4.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  7403&submit=SIMBAD+search):  HR  7403 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.130 **6.350** 6.253 6.183 6.046] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=19+27+36.386160&d=%2B37+56+28.320000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 53 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (12.000000, 2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=V923  AQL&submit=SIMBAD+search): V923  AQL (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.055 **6.085** 5.960 5.897 5.814] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=19+30+33.125760&d=%2B03+26+39.829200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 54 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-2.000000, -2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=BET 2 CYG&submit=SIMBAD+search): BET 2 CYG (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.795 **4.585** 5.208 5.258 5.258] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=19+30+45.401280&d=%2B27+57+54.907200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 55 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (2.000000, -2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 11   CYG&submit=SIMBAD+search):  11   CYG (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.900 **6.075** 6.072 6.163 6.143] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=19+35+48.310560&d=%2B36+56+40.365600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 56 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, -6.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  7482&submit=SIMBAD+search):  HR  7482 (Em*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.780 **6.365** 5.722 5.670 5.651] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=19+40+28.332960&d=%2B20+28+37.354800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 57 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (24.000000, -16.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 12   VUL&submit=SIMBAD+search):  12   VUL (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.680 **4.940** 5.388 5.051 4.947] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=19+51+4.098480&d=%2B22+36+36.201600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 58 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-2.000000, -10.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=  9   SGE&submit=SIMBAD+search):   9   SGE (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.180 **6.245** 6.110 6.166 6.154] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=19+52+21.764400&d=%2B18+40+18.606000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 59 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, -2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 25   CYG&submit=SIMBAD+search):  25   CYG (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.955 **5.200** 5.413 5.484 5.486] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=19+59+55.194720&d=%2B37+02+34.425600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 60 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-4.000000, -12.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  7678&submit=SIMBAD+search):  HR  7678 (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.000 **5.395** 4.416 4.258 4.041] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=20+04+36.174960&d=%2B32+13+6.909600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 61 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (2.000000, 12.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 28   CYG&submit=SIMBAD+search):  28   CYG (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.745 **4.990** 5.071 5.155 5.038] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=20+09+25.628880&d=%2B36+50+22.545600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 62 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (2.000000, -12.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 20   VUL&submit=SIMBAD+search):  20   VUL (Star) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.760 **5.950** 5.967 6.054 5.998] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=20+12+0.698880&d=%2B26+28+43.618800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 63 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-6.000000, -8.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=**P   CYG&submit=SIMBAD+search): **P   CYG (OpCl) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.070 **4.595** 3.624 3.409 3.053] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=20+17+47.199840&d=%2B38+01+58.584000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 64 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, -8.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 25   VUL&submit=SIMBAD+search):  25   VUL (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.390 **5.565** 5.557 5.636 5.587] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=20+22+3.427680&d=%2B24+26+45.996000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 65 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (4.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  7807&submit=SIMBAD+search):  HR  7807 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.625 **5.920** 6.174 6.228 6.165] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=20+23+44.368320&d=%2B37+28+35.115600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 66 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (2.000000, 8.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  7843&submit=SIMBAD+search):  HR  7843 (**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.820 **5.940** 5.888 5.957 5.924] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=20+29+27.122400&d=%2B56+04+5.516400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 67 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=LAM   CYG&submit=SIMBAD+search): LAM   CYG (Star) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.220 **6.285** 4.663 4.779 4.794] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=20+47+24.529440&d=%2B36+29+26.822400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 68 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-6.000000, -2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 55   CYG&submit=SIMBAD+search):  55   CYG (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.105 **4.680** 4.182 4.049 3.946] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=20+48+56.288640&d=%2B46+06+50.860800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 69 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (2.000000, 2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  7983&submit=SIMBAD+search):  HR  7983 (Em*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.240 **6.360** 6.413 6.484 6.496] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=20+49+54.633120&d=%2B46+39+40.600800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 70 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-2.000000, -6.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  8020&submit=SIMBAD+search):  HR  8020 (BlueSG**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.010 **5.495** 4.732 4.502 4.337] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=20+55+49.814400&d=%2B47+25+3.630000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 71 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, -2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  8023&submit=SIMBAD+search):  HR  8023 (SB) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.970 **5.950** 5.796 5.831 5.857] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=20+56+34.772160&d=%2B44+55+28.952400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 72 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (2.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=V832  CYG&submit=SIMBAD+search): V832  CYG (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.625 **4.780** 4.657 4.457 4.345] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=20+59+49.545360&d=%2B47+31+15.146400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 73 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-4.000000, -6.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  8153&submit=SIMBAD+search):  HR  8153 (**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.555 **6.345** 5.915 5.919 5.877] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=21+17+18.786480&d=%2B58+36+41.356800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 74 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (0.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=UPS   CYG&submit=SIMBAD+search): UPS   CYG (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.230 **4.420** 4.701 4.540 4.478] mag

Exposure Time: **5 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=21+17+55.073280&d=%2B34+53+48.663600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 75 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (6.000000, -10.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 68   CYG&submit=SIMBAD+search):  68   CYG (SB) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.940 **5.060** 5.036 5.107 5.114] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=21+18+27.184560&d=%2B43+56+45.362400&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 76 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (2.000000, 4.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=  6   CEP&submit=SIMBAD+search):   6   CEP (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.095 **5.195** 4.828 4.774 4.589] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=21+19+22.176720&d=%2B64+52+18.627600&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 77 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (2.000000, -2.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  8259&submit=SIMBAD+search):  HR  8259 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [6.120 **6.170** 6.002 6.009 6.067] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=21+34+27.461760&d=%2B51+41+54.510000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 78 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (-4.000000, -4.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  8281&submit=SIMBAD+search):  HR  8281 (**) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.845 **5.625** 5.207 5.254 5.215] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=21+38+57.606960&d=%2B57+29+20.544000&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 79 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (8.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= 16   PEG&submit=SIMBAD+search):  16   PEG (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [4.870 **5.145** 5.390 5.491 5.545] mag

Exposure Time: **10 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=21+53+3.774480&d=%2B25+55+30.493200&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---

ID: 80 

Position: 	[  ra,  dec] = [**0d51m25.758s**, **63d38m25.7568s**]

Position: 	[  ra,  dec] = [00:03:25.717200, 63:38:25.756800]

Position: 	[glon, glat] = [117.632919888, 1.26376658191]

USNO-B1.0 Epoch: 2000.0

Proper motion: (10.000000, 0.000000) +- (0.000000, 0.000000)

[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident= HR  8375&submit=SIMBAD+search):  HR  8375 (Be*) 

USNO-B1.0 [Bmag **Rmag** Jmag Hmag Ksmag ] = [5.895 **6.050** 6.046 6.141 6.159] mag

Exposure Time: **20 min**
![](https://archive.stsci.edu/cgi-bin/dss_search?v=poss2ukstu_red&r=21+55+30.941760&d=%2B65+19+14.746800&e=J2000&h=15.000000&w=15.000000&f=gif&c=none&fov=NONE&v3=)

---